#Segmentação por Instância - YOLO V10

O YOLOv10 possui uma implementação específica (fork da Ultralytics). Instalação diretamente do GitHub oficial deles.

In [ ]:
print("⚙️ Configurando ambiente...")
# Instala biblioteca necessária
try:
    import ultralytics
except ImportError:
    !pip install -q ultralytics

# Evita que o código quebre se você rodar a célula mais de uma vez
if not hasattr(torch.load, 'is_patched'):
    _original_load = torch.load
    def patched_load(*args, **kwargs):
        if 'weights_only' not in kwargs: kwargs['weights_only'] = False
        return _original_load(*args, **kwargs)
    patched_load.is_patched = True
    torch.load = patched_load

##Bibliotecas importantes

In [ ]:
import os
import shutil
import glob
import torch
from ultralytics import YOLOv10

## PREPARAÇÃO E CONVERSÃO DO DATASET
O dataset *Raccoon* original foi criado para **Detecção de Objetos** (apenas caixas/bounding boxes), mas este projeto requer **Segmentação por Instância** (máscaras/polígonos).

**O que este script faz:**
1.  **Limpeza:** Remove versões anteriores do dataset para evitar conflitos.
2.  **Conversão Matemática:** A função `convert_box_to_polygon` lê as coordenadas da caixa (`x_centro`, `y_centro`, `largura`, `altura`) e as converte matematicamente em um polígono de 4 pontos (cantos do quadrado).
    * *Resultado:* Cria "máscaras quadradas" que permitem o treinamento do modelo `-seg` sem erros.
3.  **Organização:** Estrutura as pastas em `train` e `val` conforme exigido pelo YOLO.
4.  **Configuração:** Gera automaticamente o arquivo `data.yaml` apontando para os novos dados convertidos.

In [ ]:
print("\n📂 Preparando Dataset para SEGMENTAÇÃO...")

dataset_dir = '/content/raccoon_segmentation'
if os.path.exists(dataset_dir): shutil.rmtree(dataset_dir)

# 2.1 Baixar Dataset
if not os.path.exists('/content/YOLO-Datasets'):
    !git clone https://github.com/ndb796/YOLO-Datasets.git

# 2.2 Função para converter CAIXA (Detecção) em POLÍGONO (Segmentação)
def convert_box_to_polygon(label_path, target_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 5:
            cls = parts[0]
            # Lê coordenadas da caixa (centro_x, centro_y, largura, altura)
            x, y, w, h = map(float, parts[1:5])

            # Calcula os 4 cantos da caixa para simular um polígono
            x1, y1 = x - w/2, y - h/2 # Superior esquerdo
            x2, y2 = x + w/2, y - h/2 # Superior direito
            x3, y3 = x + w/2, y + h/2 # Inferior direito
            x4, y4 = x - w/2, y + h/2 # Inferior esquerdo

            # Cria linha no formato de segmentação (pontos xy)
            new_lines.append(f"{cls} {x1:.6f} {y1:.6f} {x2:.6f} {y2:.6f} {x3:.6f} {y3:.6f} {x4:.6f} {y4:.6f}\n")

    with open(target_path, 'w') as f:
        f.writelines(new_lines)

# 2.3 Organizar pastas
for split_yolo, split_orig in [('train', 'train'), ('val', 'valid')]:
    os.makedirs(f"{dataset_dir}/images/{split_yolo}", exist_ok=True)
    os.makedirs(f"{dataset_dir}/labels/{split_yolo}", exist_ok=True)

    # Pegar imagens originais
    src_path = f"/content/YOLO-Datasets/raccoon_dataset/{split_orig}"
    images = glob.glob(f"{src_path}/images/*.jpg") + glob.glob(f"{src_path}/images/*.jpeg")

    print(f"Processando {len(images)} imagens para {split_yolo}...")

    for img in images:
        # Copia imagem
        shutil.copy(img, f"{dataset_dir}/images/{split_yolo}/")

        # Converte e cria o Label de Segmentação
        lbl_name = os.path.basename(img).rsplit('.', 1)[0] + '.txt'
        src_lbl = f"{src_path}/labels/{lbl_name}"
        if os.path.exists(src_lbl):
            convert_box_to_polygon(src_lbl, f"{dataset_dir}/labels/{split_yolo}/{lbl_name}")

##Criação do Arquivo de Configuração YAML  

O YOLO não lê as pastas diretamente de forma mágica; ele precisa deste arquivo .yaml que define explicitamente:

* Onde estão as imagens de treino (para aprender) e de validação (para se testar).

* Quantas classes existem (nc: 1).

* Os nomes dessas classes (names: ['raccoon']).

Ao criar esse arquivo via código, evita-se erros de "arquivo não encontrado".

In [ ]:
yaml_content = f"path: {dataset_dir}\ntrain: images/train\nval: images/val\nnc: 1\nnames: ['raccoon']"
with open(f"{dataset_dir}/data.yaml", 'w') as f: f.write(yaml_content)

##Treinamento (fine-tunning)

Esta é a célula principal onde o aprendizado acontece. Utilizamos uma técnica chamada Transfer Learning ao carregar os pesos yolov10n.pt. Isso significa que o modelo não começa do zero; ele já traz o conhecimento de milhões de imagens (COCO dataset) e apenas "ajusta" seus parâmetros para se especializar em detectar guaxinins.

* Epochs=50: O modelo passará 50 vezes pelo dataset completo para aprender.

* Device=0: Força o uso da GPU (essencial no Colab) para que o treino leve minutos em vez de horas.

* Best.pt: Ao final, o script indica onde o melhor modelo foi salvo, pronto para ser usado em inferências ou exportado para projetos embarcados.

In [ ]:
print("\n🚀 Iniciando Treinamento...")

# Verifica GPU (image_fca75e.png)
device = 0 if torch.cuda.is_available() else 'cpu'
if device == 'cpu': print("⚠️ AVISO: GPU não detectada! O treino será lento.")
else: print(f"✅ GPU Ativa: {torch.cuda.get_device_name(0)}")

# Usa modelo de Segmentação, usaou-se o padrão da biblioteca (v8n-seg)
model = YOLO('yolov8n-seg.pt')

try:
    results = model.train(
        data=f"{dataset_dir}/data.yaml",
        epochs=50,
        imgsz=640,
        batch=16,
        plots=True,
        device=device,
        task='segment' # Força modo segmentação
    )
    print(f"\n🏆 Sucesso! Resultados salvos em: {results.save_dir}")
except Exception as e:
    print(f"\n❌ Erro no treino: {e}")

## 📊 O que é o WandB (Weights & Biases)?

Durante o treinamento, você pode ter visto um pedido de **API Key** do **WandB**.

O **Weights & Biases (W&B)** é uma plataforma de MLOps (Machine Learning Operations) usada para rastrear experimentos. Ele funciona como um **painel de controle na nuvem** onde você pode visualizar:
* Gráficos de perda (loss) e precisão em tempo real.
* O uso de hardware (GPU/CPU/Memória).
* Comparação entre diferentes treinamentos.

**Preciso usar?**
* **Não é obrigatório.** O treinamento funciona perfeitamente sem ele.
* O YOLO utiliza o WandB por padrão para gerar gráficos bonitos na web, mas você pode usar apenas os gráficos locais que são salvos na pasta do Colab.

**Como prosseguir?**
1.  **Se quiser usar:** Crie uma conta em [wandb.ai](https://wandb.ai), pegue sua chave de API nas configurações e cole quando for pedido.
2.  **Se NÃO quiser usar (Mais rápido):** Execute a célula de código abaixo para desativar essa integração e pular o pedido de login.

In [ ]:
 # Desativa a integração com o WandB para não pedir senha/login
import os
os.environ['WANDB_MODE'] = 'disabled'

# Também garante que o YOLO não tente logar
try:
    import wandb
    wandb.init(mode="disabled")
except ImportError:
    pass

print("🚫 Integração com WandB desativada. O treino seguirá sem pedir senha.")

## 🏆 Analisando os Resultados do Treinamento

O YOLO salva automaticamente todos os resultados de cada treinamento dentro da pasta `runs`.

### 📂 Como acessar os arquivos:
1.  No menu lateral esquerdo do Colab, clique no ícone de **Pastas** (📁).
2.  Navegue até: `runs` > `detect` > `train` (ou `train2`, `train3` se tiver treinado mais vezes).

### 📊 Arquivos Importantes:
* **`results.png`**: Este é o principal gráfico. Ele mostra a evolução do erro (Loss) e da precisão (mAP).
    * *Loss (diminuindo):* O modelo está errando menos.
    * *mAP50 (aumentando):* A precisão da detecção está melhorando.
* **`confusion_matrix.png`**: Mostra onde o modelo está se confundindo (ex: achando que fundo é guaxinim ou vice-versa).
* **`val_batch0_pred.jpg`**: Mostra exemplos reais de validação com as "caixas" desenhadas pelo modelo. É a prova visual se ele aprendeu ou não.
* **`weights/best.pt`**: ⚠️ **MUITO IMPORTANTE!** Este é o arquivo do seu modelo final.
    * **Clique nos três pontinhos ao lado dele e faça o DOWNLOAD.** Se o Colab desconectar, esse arquivo some e você perde o treino.